学习资源：[单发多框检测（SSD）](http://zh.gluon.ai/chapter_computer-vision/ssd.html)

In [1]:
import time
import gluonbook as gb
from mxnet import gluon, image, nd, contrib, init, autograd
from mxnet.gluon import nn

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# SSD 模型

> 给定输入图片，其首先使用主要由卷积层组成的模块来进行特征抽取。在其输出上，我们以每个像素为中心构建多个锚框（往左的箭头），然后使用`softmax` 来对每个锚框判断其包含的物体类别，以及预测它到真实物体边界框的距离。卷积层的输出同时被输入到一个高宽减半模块（往上的箭头）来缩小图片尺寸。这个模块的输入将进入到另一个卷积模块抽取特征，和构建锚框来预测物体类别和边界框。这样设计的目的是在不同的尺寸上进行目标检测，例如之前的锚框主要检测图片中尺寸较小的物体，而这里我们则检测尺寸稍大的物体。自然的，我们会重复这一过程多次来在多种不同的尺寸下检测物体。

![](http://static.zybuluo.com/xinet/fzcupc7cl81igy91ajnp5323/ssd.png)


## 类别预测
假设数据集中有 $n$ 种不同的物体，那么我们将对每个锚框做 $n+1$ 类分类，其中类 $0$ 表示锚框只包含背景。因为我们对输入像素为中心生成多个锚框，记为 $a$，我们将会预测 $hwa$ 个锚框的分类结果，这里 $h$ 和 $w$分别是输入高和宽。如果使用正常的全连接层作为输出，可能会导致有过多的模型参数。